In [17]:
# ==========================================================
# CGAN (pix2pix) 学習: Wavelet Tensor → Clean Tensor
# 改良版：出力フォルダ構成 + 学習時間測定 + Lossグラフ保存機能付き
# ==========================================================
import os
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd


# ==========================================================
# === Dataset (Tensor入力対応) ===
# ==========================================================
class AE2CleanTensorDataset(Dataset):
    """
    AE (敵対的サンプルTensor) と Clean (正解画像Tensor) のペアを読み込むDataset。
    値が [0,1] の場合は自動で [-1,1] に正規化。
    """
    def __init__(self, ae_dir, clean_dir, auto_normalize=True):
        self.ae_dir = ae_dir
        self.clean_dir = clean_dir
        self.auto_normalize = auto_normalize
        self.filenames = sorted(os.listdir(ae_dir))

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        ae_path = os.path.join(self.ae_dir, self.filenames[idx])
        clean_path = os.path.join(self.clean_dir, self.filenames[idx])

        ae_tensor = torch.load(ae_path)
        clean_tensor = torch.load(clean_path)

        if ae_tensor.ndim == 4:
            ae_tensor = ae_tensor.squeeze(0)
        if clean_tensor.ndim == 4:
            clean_tensor = clean_tensor.squeeze(0)

        if ae_tensor.shape[0] == 1:
            ae_tensor = ae_tensor.repeat(3, 1, 1)
        if clean_tensor.shape[0] == 1:
            clean_tensor = clean_tensor.repeat(3, 1, 1)

        if self.auto_normalize:
            if ae_tensor.min() >= 0.0 and ae_tensor.max() <= 1.0:
                ae_tensor = (ae_tensor - 0.5) / 0.5
            if clean_tensor.min() >= 0.0 and clean_tensor.max() <= 1.0:
                clean_tensor = (clean_tensor - 0.5) / 0.5

        return ae_tensor, clean_tensor


# ==========================================================
# === Generator (U-Net構造) ===
# ==========================================================
class UNetGenerator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=64):
        super().__init__()
        self.encoder = nn.Sequential(
            self.block(in_channels, features, normalize=False),
            self.block(features, features * 2),
            self.block(features * 2, features * 4),
            self.block(features * 4, features * 8),
            self.block(features * 8, features * 8),
        )
        self.decoder = nn.Sequential(
            self.upblock(features * 8, features * 8),
            self.upblock(features * 8 * 2, features * 4),
            self.upblock(features * 4 * 2, features * 2),
            self.upblock(features * 2 * 2, features),
            nn.ConvTranspose2d(features * 2, out_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def block(self, in_c, out_c, normalize=True):
        layers = [nn.Conv2d(in_c, out_c, 4, 2, 1)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.LeakyReLU(0.2))
        return nn.Sequential(*layers)

    def upblock(self, in_c, out_c):
        return nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, 4, 2, 1),
            nn.BatchNorm2d(out_c),
            nn.ReLU()
        )

    def forward(self, x):
        skips = []
        for layer in self.encoder:
            x = layer(x)
            skips.append(x)
        skips = skips[:-1][::-1]
        for idx, layer in enumerate(self.decoder[:-2]):
            x = layer(x)
            if idx < len(skips):
                x = torch.cat([x, skips[idx]], 1)
        return self.decoder[-2](x)


# ==========================================================
# === Discriminator (PatchGAN) ===
# ==========================================================
class PatchDiscriminator(nn.Module):
    def __init__(self, in_channels=6, features=64):
        super().__init__()
        self.net = nn.Sequential(
            self.block(in_channels, features, normalize=False),
            self.block(features, features * 2),
            self.block(features * 2, features * 4),
            nn.Conv2d(features * 4, 1, kernel_size=4, stride=1, padding=1),
            nn.Sigmoid()
        )

    def block(self, in_c, out_c, normalize=True):
        layers = [nn.Conv2d(in_c, out_c, 4, 2, 1)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.LeakyReLU(0.2))
        return nn.Sequential(*layers)

    def forward(self, x, y):
        return self.net(torch.cat([x, y], 1))


# ==========================================================
# === 学習ループ ===
# ==========================================================
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ae_dir = r"C:\Users\sit\wavelet_CGAN\testdata3\data1_FGSM\wavelet_eval\wavelet_tensor"
    clean_dir = r"C:\Users\sit\wavelet_CGAN\testdata3\data1_clean_tensor"

    save_root = r"C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_train"
    save_img_dir = os.path.join(save_root, "images")
    save_weight_dir = os.path.join(save_root, "weights")
    save_log_dir = os.path.join(save_root, "logs")
    os.makedirs(save_img_dir, exist_ok=True)
    os.makedirs(save_weight_dir, exist_ok=True)
    os.makedirs(save_log_dir, exist_ok=True)

    dataset = AE2CleanTensorDataset(ae_dir, clean_dir)
    loader = DataLoader(dataset, batch_size=8, shuffle=True)

    gen = UNetGenerator().to(device)
    disc = PatchDiscriminator().to(device)

    opt_g = torch.optim.Adam(gen.parameters(), lr=2e-4, betas=(0.5, 0.999))
    opt_d = torch.optim.Adam(disc.parameters(), lr=2e-4, betas=(0.5, 0.999))
    bce = nn.BCELoss()
    l1 = nn.L1Loss()

    G_loss_list, D_loss_list = [], []

    print(f"\n=== CGAN (pix2pix) 学習開始 ===")
    print(f"入力: {ae_dir}")
    print(f"正解: {clean_dir}")
    print(f"出力: {save_root}\n")

    total_start = time.time()

    for epoch in range(100):
        loop = tqdm(loader, desc=f"Epoch {epoch+1}/100")
        epoch_g_loss, epoch_d_loss = 0, 0

        for ae_tensor, clean_tensor in loop:
            ae_tensor, clean_tensor = ae_tensor.to(device), clean_tensor.to(device)

            fake_img = gen(ae_tensor).detach()
            real_pred = disc(ae_tensor, clean_tensor)
            fake_pred = disc(ae_tensor, fake_img)
            real_loss = bce(real_pred, torch.ones_like(real_pred))
            fake_loss = bce(fake_pred, torch.zeros_like(fake_pred))
            d_loss = (real_loss + fake_loss) / 2
            opt_d.zero_grad()
            d_loss.backward()
            opt_d.step()

            fake_img = gen(ae_tensor)
            pred = disc(ae_tensor, fake_img)
            adv_loss = bce(pred, torch.ones_like(pred))
            l1_loss_val = l1(fake_img, clean_tensor) * 100
            g_loss = adv_loss + l1_loss_val
            opt_g.zero_grad()
            g_loss.backward()
            opt_g.step()

            epoch_g_loss += g_loss.item()
            epoch_d_loss += d_loss.item()

        G_loss_list.append(epoch_g_loss / len(loader))
        D_loss_list.append(epoch_d_loss / len(loader))

        with torch.no_grad():
            sample = (fake_img * 0.5 + 0.5).clamp(0, 1)
            save_image(sample, os.path.join(save_img_dir, f"fake_epoch{epoch+1}.png"))

        torch.save(gen.state_dict(), os.path.join(save_weight_dir, f"gen_weights_epoch{epoch+1}.pth"))

        print(f"🕒 Epoch {epoch+1} 完了")

    df_log = pd.DataFrame({"Epoch": range(1, 101), "G_loss": G_loss_list, "D_loss": D_loss_list})
    df_log.to_csv(os.path.join(save_log_dir, "loss_history.csv"), index=False)

    plt.figure(figsize=(10,5))
    plt.plot(G_loss_list, label="Generator Loss")
    plt.plot(D_loss_list, label="Discriminator Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss Curve (100 Epoch)")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(save_log_dir, "loss_curve.png"))
    plt.close()

    total_time = time.time() - total_start
    print(f"\n=== 学習完了 ===")
    print(f"⏱ 総学習時間: {total_time/60:.2f} 分")
    print(f"生成画像: {save_img_dir}")
    print(f"モデル重み: {save_weight_dir}")
    print(f"ログ: {save_log_dir}")
    print("==========================================================")


if __name__ == "__main__":
    train()



=== CGAN (pix2pix) 学習開始 ===
入力: C:\Users\sit\wavelet_CGAN\testdata3\data1_FGSM\wavelet_eval\wavelet_tensor
正解: C:\Users\sit\wavelet_CGAN\testdata3\data1_clean_tensor
出力: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_train



Epoch 1/100: 100%|██████████| 125/125 [00:05<00:00, 23.74it/s]


🕒 Epoch 1 完了


Epoch 2/100: 100%|██████████| 125/125 [00:05<00:00, 25.00it/s]


🕒 Epoch 2 完了


...

🕒 Epoch 98 完了


Epoch 99/100: 100%|██████████| 125/125 [00:05<00:00, 24.64it/s]


🕒 Epoch 99 完了


Epoch 100/100: 100%|██████████| 125/125 [00:05<00:00, 24.78it/s]


🕒 Epoch 100 完了

=== 学習完了 ===
⏱ 総学習時間: 8.75 分
生成画像: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_train\images
モデル重み: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_train\weights
ログ: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_train\logs


In [9]:
# ==========================================================
# CGAN (pix2pix) 推論 + 分類結果CSV出力
# ==========================================================
import os
import torch
import torch.nn.functional as F
from torchvision import models
from torchvision.utils import save_image
from tqdm import tqdm
import pandas as pd
from torch import nn


# --- MobileNetV2 分類器ロード ---
def load_classifier(device):
    model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
    model = model.to(device).eval()
    categories = models.MobileNet_V2_Weights.IMAGENET1K_V1.meta["categories"]
    normalize = models.MobileNet_V2_Weights.IMAGENET1K_V1.transforms()
    return model, normalize, categories


# --- Generator (U-Net) ---
class UNetGenerator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=64):
        super().__init__()
        self.encoder = nn.Sequential(
            self.block(in_channels, features, normalize=False),
            self.block(features, features * 2),
            self.block(features * 2, features * 4),
            self.block(features * 4, features * 8),
            self.block(features * 8, features * 8),
        )
        self.decoder = nn.Sequential(
            self.upblock(features * 8, features * 8),
            self.upblock(features * 8 * 2, features * 4),
            self.upblock(features * 4 * 2, features * 2),
            self.upblock(features * 2 * 2, features),
            nn.ConvTranspose2d(features * 2, out_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def block(self, in_c, out_c, normalize=True):
        layers = [nn.Conv2d(in_c, out_c, 4, 2, 1)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.LeakyReLU(0.2))
        return nn.Sequential(*layers)

    def upblock(self, in_c, out_c):
        return nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, 4, 2, 1),
            nn.BatchNorm2d(out_c),
            nn.ReLU()
        )

    def forward(self, x):
        skips = []
        for layer in self.encoder:
            x = layer(x)
            skips.append(x)
        skips = skips[:-1][::-1]
        for idx, layer in enumerate(self.decoder[:-2]):
            x = layer(x)
            if idx < len(skips):
                x = torch.cat([x, skips[idx]], 1)
        return self.decoder[-2](x)


# --- 復元＋分類処理 ---
def restore_and_classify():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    input_dir = r"C:\Users\sit\wavelet_CGAN\testdata3\data3_CW\wavelet_eval\wavelet_tensor"
    output_dir = r"C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore"
    weight_path = r"C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_train\weights\gen_weights_epoch100.pth"
    csv_path = os.path.join(output_dir, "pix2pix_class_results.csv")

    os.makedirs(output_dir, exist_ok=True)

    # --- Generatorロード ---
    gen = UNetGenerator().to(device)
    gen.load_state_dict(torch.load(weight_path, map_location=device))
    gen.eval()

    # --- 分類モデルロード ---
    clf, normalize, categories = load_classifier(device)

    results = []
    print(f"\n=== CGAN推論＋分類開始 ===\n")

    with torch.no_grad():
        for fname in tqdm(sorted(os.listdir(input_dir))):
            if not fname.endswith(".pt"):
                continue

            tensor = torch.load(os.path.join(input_dir, fname), map_location=device)
            if tensor.min() >= 0.0 and tensor.max() <= 1.0:
                tensor = (tensor - 0.5) / 0.5
            if tensor.ndim == 3:
                tensor = tensor.unsqueeze(0)

            # --- CGANによる復元 ---
            restored = gen(tensor.to(device))
            restored_01 = (restored * 0.5 + 0.5).clamp(0, 1)

            # --- 分類 ---
            normed = normalize(restored_01.squeeze(0))
            preds = F.softmax(clf(normed.unsqueeze(0)), dim=1)
            conf, idx = preds.max(1)
            label = categories[int(idx)]
            confidence = float(conf.item())

            # --- 保存 ---
            save_path = os.path.join(output_dir, fname.replace(".pt", "_restored.png"))
            save_image(restored_01.cpu(), save_path)

            results.append({
                "ImageId": os.path.splitext(fname)[0],
                "CGAN_Label": label,
                "CGAN_Confidence": round(confidence, 6),
                "Restored_Path": save_path
            })

    # --- CSV出力 ---
    pd.DataFrame(results).to_csv(csv_path, index=False, encoding="utf-8-sig")

    print(f"\n✅ 復元＆分類完了")
    print(f"出力画像フォルダ: {output_dir}")
    print(f"分類結果CSV: {csv_path}")


# --- 実行 ---
if __name__ == "__main__":
    restore_and_classify()



=== CGAN推論＋分類開始 ===



100%|██████████| 1000/1000 [00:45<00:00, 22.22it/s]


✅ 復元＆分類完了
出力画像フォルダ: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore
分類結果CSV: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore\pix2pix_class_results.csv


In [10]:
# ==========================================================
# 攻撃前＋攻撃後＋Wavelet＋CGAN比較＆成功率判定（完全版）
# ==========================================================
import pandas as pd
from pathlib import Path

print("=== 4段階CSV統合（Wavelet+CGAN対応）を開始 ===\n")

# ==========================================================
# === 各CSVパス ===
# ==========================================================
before_csv = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\data3\clean_dataset_labels.csv")
after_csv = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\data3_CW\CW_Results.csv")
wavelet_csv = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\data3_CW\wavelet_eval\CW_Wavelet_class_results.csv")
cgan_csv = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore\pix2pix_class_results.csv")

# --- 出力先 ---
output_csv = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\compare_CW_Wavelet_CGAN.csv")

# ==========================================================
# === CSV読み込み ===
# ==========================================================
print(f"攻撃前: {before_csv}")
print(f"攻撃後: {after_csv}")
print(f"分類回復後(Wavelet): {wavelet_csv}")
print(f"分類回復後(CGAN): {cgan_csv}\n")

df_before = pd.read_csv(before_csv)
df_after = pd.read_csv(after_csv)
df_wave = pd.read_csv(wavelet_csv)
df_cgan = pd.read_csv(cgan_csv)

# ==========================================================
# === ImageId整形 ===
# ==========================================================
df_before["ImageId"] = df_before["image"].apply(lambda x: Path(str(x)).stem)
df_after["ImageId"] = df_after["image"].apply(lambda x: Path(str(x)).stem)

# ✅ CGAN側のファイル名修正
# （_wavelet や _restored のついたファイルにも対応）
df_cgan["ImageId"] = df_cgan["ImageId"].apply(
    lambda x: Path(str(x)).stem.replace("_wavelet_restored", "").replace("_wavelet", "").replace("_restored", "")
)

# ==========================================================
# === 列名統一 ===
# ==========================================================
df_before = df_before.rename(columns={
    "label_name": "label_name_before",
    "confidence": "confidence_before"
})
df_after = df_after.rename(columns={
    "Advs_label_name": "label_name_after",
    "Advs_confidence": "confidence_after"
})

# ==========================================================
# === 結合処理 ===
# ==========================================================
df_merged = (
    df_before
    .merge(df_after, on="ImageId", suffixes=("_before", "_after"))
    .merge(df_wave, on="ImageId", how="left")
    .merge(df_cgan, on="ImageId", how="left")
)

# ==========================================================
# === 成功判定 ===
# ==========================================================
df_merged["AttackSuccess"] = df_merged["label_name_before"] != df_merged["label_name_after"]
df_merged["RecoverySuccess_Wavelet"] = df_merged["label_name_before"] == df_merged["Wavelet_Label"]
df_merged["RecoverySuccess_CGAN"] = df_merged["label_name_before"] == df_merged["CGAN_Label"]

# ==========================================================
# === 出力整形 ===
# ==========================================================
cols = [
    "ImageId",
    "label_name_before", "confidence_before",
    "label_name_after", "confidence_after",
    "Wavelet_Label", "Wavelet_Confidence",
    "CGAN_Label", "CGAN_Confidence",
    "AttackSuccess",
    "RecoverySuccess_Wavelet",
    "RecoverySuccess_CGAN"
]
df_final = df_merged[cols]

# ==========================================================
# === 結果出力 ===
# ==========================================================
df_final.to_csv(output_csv, index=False, encoding="utf-8-sig")

# ==========================================================
# === 統計 ===
# ==========================================================
total = len(df_final)
atk_success = df_final["AttackSuccess"].sum()
wavelet_ok = df_final["RecoverySuccess_Wavelet"].sum()
cgan_ok = df_final["RecoverySuccess_CGAN"].sum()

print("\n=== 統計結果 ===")
print(f"攻撃成功数: {atk_success}/{total} ({atk_success/total*100:.2f}%)")
print(f"Wavelet矯正成功数: {wavelet_ok}/{total} ({wavelet_ok/total*100:.2f}%)")
print(f"CGAN矯正成功数: {cgan_ok}/{total} ({cgan_ok/total*100:.2f}%)")

print(f"\n✅ 出力完了: {output_csv}")


=== 4段階CSV統合（Wavelet+CGAN対応）を開始 ===

攻撃前: C:\Users\sit\wavelet_CGAN\testdata3\data3\clean_dataset_labels.csv
攻撃後: C:\Users\sit\wavelet_CGAN\testdata3\data3_CW\CW_Results.csv
分類回復後(Wavelet): C:\Users\sit\wavelet_CGAN\testdata3\data3_CW\wavelet_eval\CW_Wavelet_class_results.csv
分類回復後(CGAN): C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore\pix2pix_class_results.csv


=== 統計結果 ===
攻撃成功数: 1000/1000 (100.00%)
Wavelet矯正成功数: 917/1000 (91.70%)
CGAN矯正成功数: 937/1000 (93.70%)

✅ 出力完了: C:\Users\sit\wavelet_CGAN\testdata3\compare_CW_Wavelet_CGAN.csv


In [ ]:
# ==========================================================
# CGAN (pix2pix) 学習: Wavelet Tensor → Clean Tensor
# CPU固定版（GPU実験と同条件・学習時間測定付き）
# ==========================================================
import os
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from tqdm import tqdm


# ==========================================================
# === Dataset (Tensor入力対応) ===
# ==========================================================
class AE2CleanTensorDataset(Dataset):
    """
    AE (敵対的サンプルTensor) と Clean (正解画像Tensor) のペアを読み込むDataset。
    値が [0,1] の場合は自動で [-1,1] に正規化。
    """
    def __init__(self, ae_dir, clean_dir, auto_normalize=True):
        self.ae_dir = ae_dir
        self.clean_dir = clean_dir
        self.auto_normalize = auto_normalize
        self.filenames = sorted(os.listdir(ae_dir))

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        ae_path = os.path.join(self.ae_dir, self.filenames[idx])
        clean_path = os.path.join(self.clean_dir, self.filenames[idx])

        ae_tensor = torch.load(ae_path, map_location="cpu")
        clean_tensor = torch.load(clean_path, map_location="cpu")

        # shape整形
        if ae_tensor.ndim == 4:
            ae_tensor = ae_tensor.squeeze(0)
        if clean_tensor.ndim == 4:
            clean_tensor = clean_tensor.squeeze(0)

        # 1ch → RGB化
        if ae_tensor.shape[0] == 1:
            ae_tensor = ae_tensor.repeat(3, 1, 1)
        if clean_tensor.shape[0] == 1:
            clean_tensor = clean_tensor.repeat(3, 1, 1)

        # 正規化 [-1,1]
        if self.auto_normalize:
            if ae_tensor.min() >= 0.0 and ae_tensor.max() <= 1.0:
                ae_tensor = (ae_tensor - 0.5) / 0.5
            if clean_tensor.min() >= 0.0 and clean_tensor.max() <= 1.0:
                clean_tensor = (clean_tensor - 0.5) / 0.5

        return ae_tensor, clean_tensor


# ==========================================================
# === Generator (U-Net構造) ===
# ==========================================================
class UNetGenerator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=64):
        super().__init__()
        self.encoder = nn.Sequential(
            self.block(in_channels, features, normalize=False),
            self.block(features, features * 2),
            self.block(features * 2, features * 4),
            self.block(features * 4, features * 8),
            self.block(features * 8, features * 8),
        )
        self.decoder = nn.Sequential(
            self.upblock(features * 8, features * 8),
            self.upblock(features * 8 * 2, features * 4),
            self.upblock(features * 4 * 2, features * 2),
            self.upblock(features * 2 * 2, features),
            nn.ConvTranspose2d(features * 2, out_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def block(self, in_c, out_c, normalize=True):
        layers = [nn.Conv2d(in_c, out_c, 4, 2, 1)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.LeakyReLU(0.2))
        return nn.Sequential(*layers)

    def upblock(self, in_c, out_c):
        return nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, 4, 2, 1),
            nn.BatchNorm2d(out_c),
            nn.ReLU()
        )

    def forward(self, x):
        skips = []
        for layer in self.encoder:
            x = layer(x)
            skips.append(x)
        skips = skips[:-1][::-1]
        for idx, layer in enumerate(self.decoder[:-2]):
            x = layer(x)
            if idx < len(skips):
                x = torch.cat([x, skips[idx]], 1)
        return self.decoder[-2](x)


# ==========================================================
# === Discriminator (PatchGAN) ===
# ==========================================================
class PatchDiscriminator(nn.Module):
    def __init__(self, in_channels=6, features=64):
        super().__init__()
        self.net = nn.Sequential(
            self.block(in_channels, features, normalize=False),
            self.block(features, features * 2),
            self.block(features * 2, features * 4),
            nn.Conv2d(features * 4, 1, kernel_size=4, stride=1, padding=1),
            nn.Sigmoid()
        )

    def block(self, in_c, out_c, normalize=True):
        layers = [nn.Conv2d(in_c, out_c, 4, 2, 1)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.LeakyReLU(0.2))
        return nn.Sequential(*layers)

    def forward(self, x, y):
        return self.net(torch.cat([x, y], 1))


# ==========================================================
# === 学習ループ（CPU固定・GPU版と同条件） ===
# ==========================================================
def train_cpu():
    device = torch.device("cpu")  # ✅ CPU固定

    # === パス設定 ===
    ae_dir = r"C:\Users\sit\wavelet_CGAN\testdata3\data1_FGSM\tensor"
    clean_dir = r"C:\Users\sit\wavelet_CGAN\testdata3\data1_clean_tensor"

    # === 出力先構成 ===
    save_root = r"C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_train_CPU_samecond"
    save_img_dir = os.path.join(save_root, "images")
    save_weight_dir = os.path.join(save_root, "weights")
    os.makedirs(save_img_dir, exist_ok=True)
    os.makedirs(save_weight_dir, exist_ok=True)

    # === データローダ ===
    dataset = AE2CleanTensorDataset(ae_dir, clean_dir)
    loader = DataLoader(dataset, batch_size=8, shuffle=True)  # ✅ GPU版と同条件

    # === モデル定義 ===
    gen = UNetGenerator().to(device)
    disc = PatchDiscriminator().to(device)

    # === 最適化と損失関数 ===
    opt_g = torch.optim.Adam(gen.parameters(), lr=2e-4, betas=(0.5, 0.999))
    opt_d = torch.optim.Adam(disc.parameters(), lr=2e-4, betas=(0.5, 0.999))
    bce = nn.BCELoss()
    l1 = nn.L1Loss()

    print(f"\n=== CGAN (pix2pix) 学習開始 [CPUモード / GPU同条件] ===")
    print(f"入力: {ae_dir}")
    print(f"正解: {clean_dir}")
    print(f"出力: {save_root}\n")

    total_start = time.time()

    for epoch in range(100):  # ✅ GPU版と同じ100エポック
        epoch_start = time.time()
        loop = tqdm(loader, desc=f"Epoch {epoch+1}/100")

        for ae_tensor, clean_tensor in loop:
            ae_tensor, clean_tensor = ae_tensor.to(device), clean_tensor.to(device)

            # --- Discriminator ---
            fake_img = gen(ae_tensor).detach()
            real_pred = disc(ae_tensor, clean_tensor)
            fake_pred = disc(ae_tensor, fake_img)
            real_loss = bce(real_pred, torch.ones_like(real_pred))
            fake_loss = bce(fake_pred, torch.zeros_like(fake_pred))
            d_loss = (real_loss + fake_loss) / 2
            opt_d.zero_grad()
            d_loss.backward()
            opt_d.step()

            # --- Generator ---
            fake_img = gen(ae_tensor)
            pred = disc(ae_tensor, fake_img)
            adv_loss = bce(pred, torch.ones_like(pred))
            l1_loss_val = l1(fake_img, clean_tensor) * 100
            g_loss = adv_loss + l1_loss_val
            opt_g.zero_grad()
            g_loss.backward()
            opt_g.step()

            loop.set_postfix(G_loss=g_loss.item(), D_loss=d_loss.item())

        # --- 出力保存 ---
        with torch.no_grad():
            sample = (fake_img * 0.5 + 0.5).clamp(0, 1)
            save_image(sample, os.path.join(save_img_dir, f"fake_epoch{epoch+1}.png"))
        torch.save(gen.state_dict(), os.path.join(save_weight_dir, f"gen_weights_epoch{epoch+1}.pth"))

        epoch_time = time.time() - epoch_start
        print(f"🕒 Epoch {epoch+1} 完了: {epoch_time/60:.2f} 分")

    total_time = time.time() - total_start
    print(f"\n=== 学習完了（CPU / GPU同条件） ===")
    print(f"⏱ 総学習時間: {total_time/60:.2f} 分 ({total_time/3600:.2f} 時間)")
    print(f"生成画像: {save_img_dir}")
    print(f"モデル重み: {save_weight_dir}")
    print("==========================================================")


# ==========================================================
# === 実行 ===
# ==========================================================
if __name__ == "__main__":
    train_cpu()



=== CGAN (pix2pix) 学習開始 [CPUモード / GPU同条件] ===
入力: C:\Users\sit\wavelet_CGAN\testdata3\data1_FGSM\tensor
正解: C:\Users\sit\wavelet_CGAN\testdata3\data1_clean_tensor
出力: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_train_CPU_samecond



Epoch 1/100: 100%|██████████| 125/125 [01:44<00:00,  1.20it/s, D_loss=0.488, G_loss=14.5]


🕒 Epoch 1 完了: 1.74 分


Epoch 2/100: 100%|██████████| 125/125 [01:41<00:00,  1.23it/s, D_loss=0.542, G_loss=7.53]


🕒 Epoch 2 完了: 1.69 分


...

Epoch 58/100: 100%|██████████| 125/125 [01:41<00:00,  1.23it/s, D_loss=0.073, G_loss=4.54] 


🕒 Epoch 58 完了: 1.70 分


Epoch 59/100: 100%|██████████| 125/125 [01:41<00:00,  1.23it/s, D_loss=0.467, G_loss=6.64] 


🕒 Epoch 59 完了: 1.70 分


Epoch 60/100:  20%|██        | 25/125 [00:20<01:21,  1.23it/s, D_loss=0.102, G_loss=4.01] 

In [44]:
# ==========================================================
# Wavelet＋CGAN復元＋4段階比較＆成功率判定（レガシー互換：Resize/Crop含む transforms() を既定ON）
# ==========================================================
import os
import sys
import torch
import torch.nn.functional as F
import pandas as pd
from pathlib import Path
from torchvision import models
from torchvision.utils import save_image
from torch import nn
from tqdm import tqdm

# ========= 0) 再現性（任意） =========
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ========= 1) 設定：ここだけ変える =========
dataset     = "data2"        # "data1" / "data2" / "data3"
attack_type = "FGSM"         # "FGSM" / "CW"
LEGACY_PREPROCESS = True     # ✅ 前回と同じ transforms() を使う（Resize/Crop含む）

BASE = Path(r"C:\Users\sit\wavelet_CGAN\testdata3")

# 入力/出力パス（前回の構成に準拠）
clean_dir    = BASE / dataset
adv_dir      = BASE / f"{dataset}_{attack_type}"
wavelet_dir  = adv_dir / "wavelet_eval"
tensor_dir   = wavelet_dir / "wavelet_tensor"  # *_wavelet.pt

# CGAN出力（dataset+attackで整理）
cgan_out_dir = BASE / "pix2pix_restore" / f"{dataset}_{attack_type}"
cgan_out_dir.mkdir(parents=True, exist_ok=True)

# CSVパス（前回の命名に極力合わせる）
before_csv   = clean_dir / "clean_dataset_labels.csv"
after_csv    = adv_dir / f"{attack_type}_Results.csv"
wavelet_csv  = wavelet_dir / f"{attack_type}_Wavelet_class_results.csv"
cgan_csv     = cgan_out_dir / f"pix2pix_class_results_{dataset}_{attack_type}.csv"
merged_csv   = BASE / f"compare_{dataset}_{attack_type}_Wavelet_CGAN.csv"

# CGAN重み（前回パスに準拠）
weight_path  = BASE / "pix2pix_train_v2" / "weights" / "gen_weights_epoch300.pth"

# ========= 2) 事前チェック =========
def _must_exist(p: Path, hint: str = ""):
    if not p.exists():
        print(f"[ERROR] Not found: {p}\n{('  Hint: '+hint) if hint else ''}")
        sys.exit(1)

_must_exist(before_csv,  "clean_dataset_labels.csv を作成してください。")
_must_exist(after_csv,   f"{attack_type}_Results.csv を作成してください。")
_must_exist(wavelet_csv, f"{attack_type}_Wavelet_class_results.csv を作成してください。")
_must_exist(tensor_dir,  "wavelet_eval/wavelet_tensor に *_wavelet.pt を出力してください。")
_must_exist(weight_path, "pix2pix 学習済み重みのパスを確認してください。")
if not any(str(p).endswith(".pt") for p in tensor_dir.iterdir()):
    print(f"[ERROR] No .pt under: {tensor_dir}（*_wavelet.pt 必須）")
    sys.exit(1)

# ========= 3) MobileNetV2（レガシー互換前処理オプション） =========
def load_classifier(device):
    weights = models.MobileNet_V2_Weights.IMAGENET1K_V1
    model = models.mobilenet_v2(weights=weights).to(device).eval()
    categories = weights.meta["categories"]
    if LEGACY_PREPROCESS:
        # ✅ 前回と同じ：Resize(256)→CenterCrop(224)→ToTensor→Normalize
        preprocess = weights.transforms()
    else:
        # Normalizeのみ（解像度据え置き）に切り替えたい場合はこちら
        from torchvision import transforms as T
        preprocess = T.Normalize(mean=weights.meta["mean"], std=weights.meta["std"])
    return model, preprocess, categories

# ========= 4) CGAN Generator =========
class UNetGenerator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=64):
        super().__init__()
        self.encoder = nn.Sequential(
            self.block(in_channels, features, normalize=False),
            self.block(features, features * 2),
            self.block(features * 2, features * 4),
            self.block(features * 4, features * 8),
            self.block(features * 8, features * 8),
        )
        self.decoder = nn.Sequential(
            self.upblock(features * 8, features * 8),
            self.upblock(features * 8 * 2, features * 4),
            self.upblock(features * 4 * 2, features * 2),
            self.upblock(features * 2 * 2, features),
            nn.ConvTranspose2d(features * 2, out_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def block(self, in_c, out_c, normalize=True):
        layers = [nn.Conv2d(in_c, out_c, 4, 2, 1)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.LeakyReLU(0.2))
        return nn.Sequential(*layers)

    def upblock(self, in_c, out_c):
        return nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, 4, 2, 1),
            nn.BatchNorm2d(out_c),
            nn.ReLU()
        )

    def forward(self, x):
        skips = []
        for layer in self.encoder:
            x = layer(x); skips.append(x)
        skips = skips[:-1][::-1]
        for i, layer in enumerate(self.decoder[:-2]):
            x = layer(x)
            if i < len(skips):
                x = torch.cat([x, skips[i]], 1)
        return self.decoder[-2](x)

# ========= 5) CGAN復元＋分類 → CSV =========
def restore_and_classify():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    gen = UNetGenerator().to(device)
    gen.load_state_dict(torch.load(weight_path, map_location=device))
    gen.eval()

    clf, preprocess, categories = load_classifier(device)
    results = []
    print(f"\n=== CGAN推論＋分類開始 ({dataset}/{attack_type}) | LEGACY_PREPROCESS={LEGACY_PREPROCESS} ===\n")

    with torch.no_grad():
        for fname in tqdm(sorted(os.listdir(tensor_dir))):
            if not fname.endswith(".pt"):
                continue
            t = torch.load(tensor_dir / fname, map_location=device)  # [C,H,W] or [1,C,H,W] in [0,1]
            if t.ndim == 3:
                t = t.unsqueeze(0)
            # Wavelet後テンソル[0,1] → CGAN入力は[-1,1]
            t_in = (t - 0.5) / 0.5 if (t.min() >= 0 and t.max() <= 1) else t

            restored = gen(t_in.to(device))
            restored_01 = (restored * 0.5 + 0.5).clamp(0, 1)  # [0,1]

            # 分類：レガシー互換の場合は transforms() をそのまま適用
            # transforms() は PIL/Tensor 両対応＆内部でToTensor/Normalize等を処理
            x = restored_01.squeeze(0).cpu()
            x = preprocess(x).unsqueeze(0).to(device)
            preds = F.softmax(clf(x), dim=1)
            conf, idx = preds.max(1)
            label = categories[int(idx)]
            confidence = float(conf.item())

            base = Path(fname).stem.replace("_wavelet", "")
            out_png = cgan_out_dir / f"{base}_{attack_type}_restored.png"
            save_image(restored_01.cpu(), out_png)

            results.append({
                "ImageId": base,
                "CGAN_Label": label,
                "CGAN_Confidence": round(confidence, 6),
            })

    pd.DataFrame(results).to_csv(cgan_csv, index=False, encoding="utf-8-sig")
    print(f"✅ CGAN分類結果CSV: {cgan_csv}")
    print(f"✅ 復元画像: {cgan_out_dir}")

# ========= 6) 4段階CSV統合＋成功率 =========
def merge_and_evaluate():
    print(f"\n=== 4段階CSV統合＆成功率 ({dataset}/{attack_type}) ===\n")
    df_b  = pd.read_csv(before_csv)
    df_a  = pd.read_csv(after_csv)
    df_w  = pd.read_csv(wavelet_csv)
    df_g  = pd.read_csv(cgan_csv)

    df_b["ImageId"] = df_b["image"].apply(lambda x: Path(str(x)).stem)
    df_a["ImageId"] = df_a["image"].apply(lambda x: Path(str(x)).stem)
    df_g["ImageId"] = df_g["ImageId"].astype(str).apply(
        lambda s: Path(s).stem.replace(f"_{attack_type}_restored","").replace("_restored","").replace("_wavelet","")
    )

    df_b = df_b.rename(columns={"label_name":"label_name_before","confidence":"confidence_before"})
    df_a = df_a.rename(columns={"Advs_label_name":"label_name_after","Advs_confidence":"confidence_after"})

    df = (df_b.merge(df_a, on="ImageId", how="inner")
              .merge(df_w, on="ImageId", how="left")
              .merge(df_g, on="ImageId", how="left"))

    df["AttackSuccess"]            = df["label_name_before"] != df["label_name_after"]
    df["RecoverySuccess_Wavelet"]  = df["label_name_before"] == df.get("Wavelet_Label")
    df["RecoverySuccess_CGAN"]     = df["label_name_before"] == df.get("CGAN_Label")

    df.to_csv(merged_csv, index=False, encoding="utf-8-sig")

    total = len(df)
    atk   = int(df["AttackSuccess"].sum())
    w_ok  = int(df["RecoverySuccess_Wavelet"].sum())
    g_ok  = int(df["RecoverySuccess_CGAN"].sum())
    pct   = lambda n: (n/total*100.0) if total else 0.0

    print("=== 統計結果 ===")
    print(f"総数: {total}")
    print(f"攻撃成功数:        {atk}/{total} ({pct(atk):.2f}%)")
    print(f"Wavelet矯正成功数: {w_ok}/{total} ({pct(w_ok):.2f}%)")
    print(f"CGAN矯正成功数:   {g_ok}/{total} ({pct(g_ok):.2f}%)")
    print(f"\n✅ 出力: {merged_csv}")

# ========= 7) 実行 =========
if __name__ == "__main__":
    restore_and_classify()
    merge_and_evaluate()



=== CGAN推論＋分類開始 (data2/FGSM) | LEGACY_PREPROCESS=True ===



100%|██████████| 1000/1000 [00:49<00:00, 20.41it/s]

✅ CGAN分類結果CSV: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore\data2_FGSM\pix2pix_class_results_data2_FGSM.csv
✅ 復元画像: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore\data2_FGSM

=== 4段階CSV統合＆成功率 (data2/FGSM) ===

=== 統計結果 ===
総数: 1000
攻撃成功数:        1000/1000 (100.00%)
Wavelet矯正成功数: 810/1000 (81.00%)
CGAN矯正成功数:   945/1000 (94.50%)

✅ 出力: C:\Users\sit\wavelet_CGAN\testdata3\compare_data2_FGSM_Wavelet_CGAN.csv


In [1]:
# ==========================================================
# CGAN (pix2pix) 学習: Wavelet Tensor → Clean Tensor
# ★ MobileNetV2 Feature Matching Loss 導入版 (KeyError 修正済み) ★
# ==========================================================
import os
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from torchvision import models
from torch.nn.functional import l1_loss

# ==========================================================
# === ハイパーパラメータ設定 ===
# ==========================================================
LAMBDA_L1 = 100
LAMBDA_FEATURE = 0
NUM_EPOCHS = 300
LEARNING_RATE = 2e-4    

# ==========================================================
# === Dataset (Tensor入力対応) ===
# ==========================================================
class AE2CleanTensorDataset(Dataset):
    """
    AE (敵対的サンプルTensor) と Clean (正解画像Tensor) のペアを読み込むDataset。
    値が [0,1] の場合は自動で [-1,1] に正規化。
    """
    def __init__(self, ae_dir, clean_dir, auto_normalize=True):
        self.ae_dir = ae_dir
        self.clean_dir = clean_dir
        self.auto_normalize = auto_normalize
        self.filenames = sorted(os.listdir(ae_dir))

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        ae_path = os.path.join(self.ae_dir, self.filenames[idx])
        clean_path = os.path.join(self.clean_dir, self.filenames[idx])

        ae_tensor = torch.load(ae_path)
        clean_tensor = torch.load(clean_path)

        if ae_tensor.ndim == 4:
            ae_tensor = ae_tensor.squeeze(0)
        if clean_tensor.ndim == 4:
            clean_tensor = clean_tensor.squeeze(0)

        if ae_tensor.shape[0] == 1:
            ae_tensor = ae_tensor.repeat(3, 1, 1)
        if clean_tensor.shape[0] == 1:
            clean_tensor = clean_tensor.repeat(3, 1, 1)

        if self.auto_normalize:
            # [0, 1] -> [-1, 1] に正規化
            if ae_tensor.min() >= 0.0 and ae_tensor.max() <= 1.0:
                ae_tensor = (ae_tensor - 0.5) / 0.5
            if clean_tensor.min() >= 0.0 and clean_tensor.max() <= 1.0:
                clean_tensor = (clean_tensor - 0.5) / 0.5

        return ae_tensor, clean_tensor


# ==========================================================
# === Generator (U-Net構造) ===
# ==========================================================
class UNetGenerator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=64):
        super().__init__()
        self.encoder = nn.Sequential(
            self.block(in_channels, features, normalize=False),
            self.block(features, features * 2),
            self.block(features * 2, features * 4),
            self.block(features * 4, features * 8),
            self.block(features * 8, features * 8),
        )
        self.decoder = nn.Sequential(
            self.upblock(features * 8, features * 8),
            self.upblock(features * 8 * 2, features * 4),
            self.upblock(features * 4 * 2, features * 2),
            self.upblock(features * 2 * 2, features),
            nn.ConvTranspose2d(features * 2, out_channels, 4, 2, 1),
            nn.Tanh(),
        )

    def block(self, in_c, out_c, normalize=True):
        layers = [nn.Conv2d(in_c, out_c, 4, 2, 1)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.LeakyReLU(0.2))
        return nn.Sequential(*layers)

    def upblock(self, in_c, out_c):
        return nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, 4, 2, 1),
            nn.BatchNorm2d(out_c),
            nn.ReLU()
        )

    def forward(self, x):
        skips = []
        for layer in self.encoder:
            x = layer(x)
            skips.append(x)

        skips = skips[:-1][::-1]

        for idx, layer in enumerate(self.decoder[:-2]):
            x = layer(x)
            if idx < len(skips):
                x = torch.cat([x, skips[idx]], 1)

        return self.decoder[-2](x)  # ConvTranspose


# ==========================================================
# === Discriminator (PatchGAN) ===
# ==========================================================
class PatchDiscriminator(nn.Module):
    def __init__(self, in_channels=6, features=64): # 入力は Input(3ch) + Target/Fake(3ch) で 6ch
        super().__init__()
        self.net = nn.Sequential(
            self.block(in_channels, features, normalize=False),
            self.block(features, features * 2),
            self.block(features * 2, features * 4),
            # 最終層
            nn.Conv2d(features * 4, 1, kernel_size=4, stride=1, padding=1),
            nn.Sigmoid() # 0-1の確率でパッチが本物か判定
        )

    def block(self, in_c, out_c, normalize=True):
        layers = [nn.Conv2d(in_c, out_c, 4, 2, 1)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.LeakyReLU(0.2))
        return nn.Sequential(*layers)

    def forward(self, x, y):
        return self.net(torch.cat([x, y], 1))


# ==========================================================
# === Feature Extractor (MobileNetV2) ===
# ==========================================================
class MobileNetFeatureExtractor(nn.Module):
    """
    MobileNetV2をロードし、指定した層から特徴量を抽出する。
    分類器の「目」をGeneratorに組み込むために使用。
    """
    def __init__(self, device):
        super().__init__()
        
        weights = models.MobileNet_V2_Weights.IMAGENET1K_V1
        
        # ImageNet学習済みのMobileNetV2をロードし、重みを固定 (evalモード)
        mobilenet = models.mobilenet_v2(weights=weights).to(device).eval()
        
        # Generatorに直接分類器の重みを学習させたくないため、パラメータを固定
        for param in mobilenet.parameters():
            param.requires_grad = False
        
        # 特徴量抽出に使用する層 (中間層として features[:14] までを使用)
        self.feature_extractor = mobilenet.features[:14] 
        
        # ★ 修正: KeyErrorを避けるため、ImageNet標準のmeanとstdを直接設定
        IMAGENET_MEAN = [0.485, 0.456, 0.406]
        IMAGENET_STD = [0.229, 0.224, 0.225]
        
        self.mean = torch.tensor(IMAGENET_MEAN).view(1, 3, 1, 1).to(device)
        self.std = torch.tensor(IMAGENET_STD).view(1, 3, 1, 1).to(device)
        
    def forward(self, x_01):
        """
        入力: [0, 1] の範囲の画像テンソル ([B, C, H, W])
        """
        # MobileNetの標準的な前処理（正規化）
        x_norm = (x_01 - self.mean) / self.std
        
        # 特徴量を抽出
        features = self.feature_extractor(x_norm)
        return features


# ==========================================================
# === 学習ループ ===
# ==========================================================
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # === パス設定 ===
    ae_dir = r"C:\Users\sit\wavelet_CGAN\testdata3\data1_FGSM\tensor"
    clean_dir = r"C:\Users\sit\wavelet_CGAN\testdata3\data1_clean_tensor"
    
    # ★ フォルダ名にFeature Lossの重みを追加
    save_root = r"C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_train_FGSM"
    save_img_dir = os.path.join(save_root, "images")
    save_weight_dir = os.path.join(save_root, "weights")
    save_log_dir = os.path.join(save_root, "logs")
    os.makedirs(save_img_dir, exist_ok=True)
    os.makedirs(save_weight_dir, exist_ok=True)
    os.makedirs(save_log_dir, exist_ok=True)

    dataset = AE2CleanTensorDataset(ae_dir, clean_dir)
    loader = DataLoader(dataset, batch_size=8, shuffle=True)

    gen = UNetGenerator().to(device)
    disc = PatchDiscriminator().to(device)
    feature_extractor = MobileNetFeatureExtractor(device) # ★ MobileNetV2の特徴量抽出器

    opt_g = torch.optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
    opt_d = torch.optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
    bce = nn.BCELoss()
    l1 = nn.L1Loss()

    # ★ ログリストに Feature Loss を追加
    G_loss_list, D_loss_list = [], []
    Adv_loss_list, L1_loss_list = [], []
    Feature_loss_list = []

    print(f"\n=== CGAN (pix2pix) 学習開始 (MobileNet Feature Match) ===")
    print(f"★ L1 Loss 重み (LAMBDA_L1): {LAMBDA_L1}")
    print(f"★ Feature Loss 重み (LAMBDA_FEATURE): {LAMBDA_FEATURE}") # ★ 表示
    print(f"入力: {ae_dir}")
    print(f"出力: {save_root}\n")

    total_start = time.time()

    # --- Utility: [-1, 1] -> [0, 1] ---
    def to_01(t):
        return (t * 0.5 + 0.5).clamp(0, 1)

    # --- Clean Tensorの特徴量を事前に計算 ---
    clean_features_list = []
    print("Pre-calculating Clean Image Features...")
    with torch.no_grad():
        for _, clean_tensor in tqdm(loader):
            # Clean Tensor [-1, 1] を MobileNetV2 の入力範囲 [0, 1] に戻してから特徴量を抽出
            clean_features = feature_extractor(to_01(clean_tensor.to(device)))
            clean_features_list.append(clean_features)
    
    print("Pre-calculation finished.\n")
    # ----------------------------------------------------


    for epoch in range(NUM_EPOCHS):
        loop = tqdm(loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
        epoch_g_loss, epoch_d_loss = 0, 0
        epoch_adv_loss, epoch_l1_loss = 0, 0
        epoch_feature_loss = 0 # ★ 追加

        # Clean Feature Iteratorをリセット
        clean_features_iter = iter(clean_features_list)

        for ae_tensor, clean_tensor in loop:
            ae_tensor, clean_tensor = ae_tensor.to(device), clean_tensor.to(device)
            current_clean_features = next(clean_features_iter) # 事前計算した特徴量を取得

            # --- Discriminatorの学習 ---
            fake_img = gen(ae_tensor).detach()
            real_pred = disc(ae_tensor, clean_tensor)
            fake_pred = disc(ae_tensor, fake_img)
            real_loss = bce(real_pred, torch.ones_like(real_pred))
            fake_loss = bce(fake_pred, torch.zeros_like(fake_pred))
            d_loss = (real_loss + fake_loss) / 2
            opt_d.zero_grad()
            d_loss.backward()
            opt_d.step()

            # --- Generatorの学習 ---
            fake_img = gen(ae_tensor)
            
            # 1. Adversarial Loss
            pred = disc(ae_tensor, fake_img)
            adv_loss = bce(pred, torch.ones_like(pred))
            
            # 2. L1 Loss
            l1_loss_val = l1(fake_img, clean_tensor) * LAMBDA_L1
            
            # 3. ★ Feature Matching Loss
            fake_img_01 = to_01(fake_img)
            fake_features = feature_extractor(fake_img_01)
            
            # 特徴量マップ間のL1距離を計算 (Feature Matching Loss)
            feature_loss_val = l1_loss(fake_features, current_clean_features) * LAMBDA_FEATURE
            
            # Total Generator Loss
            g_loss = adv_loss + l1_loss_val + feature_loss_val
            
            opt_g.zero_grad()
            g_loss.backward()
            opt_g.step()

            # --- ロギング用の累積 ---
            epoch_g_loss += g_loss.item()
            epoch_d_loss += d_loss.item()
            epoch_adv_loss += adv_loss.item()
            epoch_l1_loss += l1_loss_val.item()
            epoch_feature_loss += feature_loss_val.item()

        # --- エポック平均の記録 ---
        G_loss_list.append(epoch_g_loss / len(loader))
        D_loss_list.append(epoch_d_loss / len(loader))
        Adv_loss_list.append(epoch_adv_loss / len(loader))
        L1_loss_list.append(epoch_l1_loss / len(loader))
        Feature_loss_list.append(epoch_feature_loss / len(loader))

        # --- 画像/重み保存 ---
        with torch.no_grad():
            sample = to_01(fake_img)
            save_image(sample, os.path.join(save_img_dir, f"fake_epoch{epoch+1}.png"))

        torch.save(gen.state_dict(), os.path.join(save_weight_dir, f"gen_weights_epoch{epoch+1}.pth"))

        print(f"🕒 Epoch {epoch+1} 完了 | G_Loss: {G_loss_list[-1]:.4f} (Adv:{Adv_loss_list[-1]:.4f}, L1:{L1_loss_list[-1]:.4f}, Feat:{Feature_loss_list[-1]:.4f}) | D_Loss: {D_loss_list[-1]:.4f}")

    # --- ログ/グラフ保存 ---
    df_log = pd.DataFrame({
        "Epoch": range(1, NUM_EPOCHS + 1),
        "G_loss": G_loss_list,
        "Adv_loss": Adv_loss_list,
        "L1_loss": L1_loss_list,
        "Feature_loss": Feature_loss_list,
        "D_loss": D_loss_list
    })
    df_log.to_csv(os.path.join(save_log_dir, "loss_history.csv"), index=False)

    plt.figure(figsize=(12,6))
    plt.plot(G_loss_list, label="Generator Total Loss", linewidth=2)
    plt.plot(Adv_loss_list, label="Adv Loss", linestyle='--', alpha=0.7)
    plt.plot(L1_loss_list, label="L1 Loss (Weighted)", linestyle='--', alpha=0.7)
    plt.plot(Feature_loss_list, label="Feature Loss (Weighted)", linestyle='--', alpha=0.7)
    plt.plot(D_loss_list, label="Discriminator Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"Training Loss Curve (L1={LAMBDA_L1}, Feat={LAMBDA_FEATURE})")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(save_log_dir, "loss_curve.png"))
    plt.close()

    total_time = time.time() - total_start
    print(f"\n=== 学習完了 ===")
    print(f"⏱ 総学習時間: {total_time/60:.2f} 分")
    print(f"出力フォルダ: {save_root}")
    print("==========================================================")


if __name__ == "__main__":
    train()


=== CGAN (pix2pix) 学習開始 (MobileNet Feature Match) ===
★ L1 Loss 重み (LAMBDA_L1): 100
★ Feature Loss 重み (LAMBDA_FEATURE): 0
入力: C:\Users\sit\wavelet_CGAN\testdata3\data1_FGSM\tensor
出力: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_train_FGSM

Pre-calculating Clean Image Features...


100%|██████████| 125/125 [00:07<00:00, 17.07it/s]


Pre-calculation finished.



Epoch 1/300:  95%|█████████▌| 119/125 [00:07<00:00, 16.92it/s]


KeyboardInterrupt: 

In [17]:
import pandas as pd
from torchvision import models
from docx import Document
from docx.shared import Inches, Pt

# === 設定 ===
csv_path = r"C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore_FGSM_train\data3_CW\compare_data3_CW_Wavelet_CGAN.csv"
output_docx = r"C:\Users\sit\wavelet_CGAN\testdata3\attack_compare_top100.docx"
num_rows = 20  # 出力件数

# === ImageNetクラス名リスト ===
weights = models.MobileNet_V2_Weights.IMAGENET1K_V1
imagenet_labels = weights.meta["categories"]

# === CSV読み込み ===
df = pd.read_csv(csv_path)

# === class_XXX → ラベル名に変換関数 ===
def convert_class_to_label(x):
    if pd.isna(x):
        return x
    if isinstance(x, str) and x.startswith("class_"):
        try:
            idx = int(x.replace("class_", "").strip())
            return imagenet_labels[idx]
        except:
            return x
    return x

# 対象列を変換（攻撃後・CGAN・Waveletなど）
for col in ["label_name_after", "Wavelet_Label", "CGAN_Label"]:
    if col in df.columns:
        df[col] = df[col].apply(convert_class_to_label)

# === 必要列だけ抽出 ===
cols = [
    "label_name_before", "confidence_before",
    "label_name_after", "confidence_after",
    "CGAN_Label", "CGAN_Confidence"
]
df_show = df[cols].head(num_rows).copy()

# === confidence を小数6桁に丸める ===
for col in ["confidence_before", "confidence_after", "CGAN_Confidence"]:
    df_show[col] = df_show[col].round(6)

# === 列名を日本語に変更 ===
japanese_columns = {
    "label_name_before": "入力画像ラベル",
    "confidence_before": "confidence（入力）",
    "label_name_after": "攻撃後ラベル",
    "confidence_after": "confidence（攻撃後）",
    "CGAN_Label": "CGAN復元後ラベル",
    "CGAN_Confidence": "confidence（復元後）"
}
df_show.rename(columns=japanese_columns, inplace=True)

# === Word 出力 ===
doc = Document()
doc.add_heading("敵対的サンプル比較表（上位20件）", level=1)
doc.add_paragraph("通常画像 → 攻撃後画像 → Wavelet+CGAN復元後")

table = doc.add_table(rows=1, cols=len(df_show.columns))
table.style = "Table Grid"

# ヘッダ行
hdr_cells = table.rows[0].cells
for i, col in enumerate(df_show.columns):
    hdr_cells[i].text = col

# データ行
for _, row in df_show.iterrows():
    row_cells = table.add_row().cells
    for i, col in enumerate(df_show.columns):
        row_cells[i].text = str(row[col])

# === 保存 ===
doc.save(output_docx)
print(f"✅ Wordファイルを出力しました: {output_docx}")


✅ Wordファイルを出力しました: C:\Users\sit\wavelet_CGAN\testdata3\attack_compare_top100.docx


In [6]:
# ==========================================================
# Wavelet＋CGAN復元＋4段階比較＆成功率判定（レガシー互換：Resize/Crop含む transforms() を既定ON）
# + Wavelet矯正失敗 / CGAN矯正失敗 画像の自動仕分け保存
# + すべてのCSV（CGAN分類結果・統合結果・失敗一覧）は「画像と同じフォルダ（cgan_out_dir）」に出力
# ==========================================================
import os
import sys
import torch
import torch.nn.functional as F
import pandas as pd
from pathlib import Path
from torchvision import models
from torchvision.utils import save_image
from torch import nn
from tqdm import tqdm
from shutil import copy2

# ========= 0) 再現性（任意） =========
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ========= 1) 設定：ここだけ変える =========
dataset     = "data3"        # "data1" / "data2" / "data3"
attack_type = "CW"         # "FGSM" / "CW"
LEGACY_PREPROCESS = True     # ✅ transforms() を使う（Resize/Crop含む前回互換）

BASE = Path(r"C:\Users\sit\wavelet_CGAN\testdata3")

# ========= 入出力パス =========
clean_dir    = BASE / dataset
adv_dir      = BASE / f"{dataset}_{attack_type}"
wavelet_dir  = adv_dir / "wavelet_eval"
tensor_dir   = wavelet_dir / "wavelet_tensor"
wavelet_png_dir = wavelet_dir / "wavelet_png"

# 復元画像の出力先（＝本スクリプトの「主フォルダ」）
cgan_out_dir = BASE / "pix2pix_restore_train_v2" / f"{dataset}_{attack_type}"
cgan_out_dir.mkdir(parents=True, exist_ok=True)

# CSVは全部このフォルダ（＝画像と同じ）へ
cgan_csv   = cgan_out_dir / f"pix2pix_class_results_{dataset}_{attack_type}.csv"
merged_csv = cgan_out_dir / f"compare_{dataset}_{attack_type}_Wavelet_CGAN.csv"

# 入力CSV（参照のみ）
before_csv   = clean_dir / "clean_dataset_labels.csv"
after_csv    = adv_dir / f"{attack_type}_Results.csv"
wavelet_csv  = wavelet_dir / f"{attack_type}_Wavelet_class_results.csv"

# ✅ 失敗画像の出力先（画像は従来どおり各フォルダへ仕分け）
wavelet_failed_dir = BASE / "wavelet_restore_failed" / f"{dataset}_{attack_type}"
cgan_failed_dir    = BASE / "cgan_restore_failed"    / f"{dataset}_{attack_type}"
wavelet_failed_dir.mkdir(parents=True, exist_ok=True)
cgan_failed_dir.mkdir(parents=True, exist_ok=True)

# 失敗一覧CSVも画像と同じフォルダ（cgan_out_dir）へ
# → 参照・共有のしやすさを優先
failed_list_csv = cgan_out_dir / f"failed_lists_{dataset}_{attack_type}.csv"

# 学習済みGeneratorの重み
weight_path  = BASE / "pix2pix_train_v2" / "weights" / "gen_weights_epoch300.pth"

# ========= 2) 事前チェック =========
def _must_exist(p: Path, hint: str = ""):
    if not p.exists():
        print(f"[ERROR] Not found: {p}")
        if hint:
            print("Hint:", hint)
        sys.exit(1)

_must_exist(before_csv,  "clean_dataset_labels.csv を作成してください。")
_must_exist(after_csv,   f"{attack_type}_Results.csv を作成してください。")
_must_exist(wavelet_csv, f"{attack_type}_Wavelet_class_results.csv を作成してください。")
_must_exist(tensor_dir,  "wavelet_eval/wavelet_tensor に *_wavelet.pt を出力してください。")
_must_exist(weight_path, "pix2pix 学習済み重みのパスを確認してください。")

if not any(str(p).endswith(".pt") for p in tensor_dir.iterdir()):
    print(f"[ERROR] No .pt under: {tensor_dir}")
    sys.exit(1)

# ========= 3) MobileNetV2（レガシー互換前処理オプション） =========
def load_classifier(device):
    weights = models.MobileNet_V2_Weights.IMAGENET1K_V1
    model = models.mobilenet_v2(weights=weights).to(device).eval()
    categories = weights.meta["categories"]
    if LEGACY_PREPROCESS:
        preprocess = weights.transforms()
    else:
        from torchvision import transforms as T
        preprocess = T.Normalize(mean=weights.meta["mean"], std=weights.meta["std"])
    return model, preprocess, categories

# ========= 4) CGAN Generator =========
class UNetGenerator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=64):
        super().__init__()
        self.encoder = nn.Sequential(
            self.block(in_channels, features, normalize=False),
            self.block(features, features * 2),
            self.block(features * 2, features * 4),
            self.block(features * 4, features * 8),
            self.block(features * 8, features * 8),
        )
        self.decoder = nn.Sequential(
            self.upblock(features * 8, features * 8),
            self.upblock(features * 8 * 2, features * 4),
            self.upblock(features * 4 * 2, features * 2),
            self.upblock(features * 2 * 2, features),
            nn.ConvTranspose2d(features * 2, out_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def block(self, in_c, out_c, normalize=True):
        layers = [nn.Conv2d(in_c, out_c, 4, 2, 1)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.LeakyReLU(0.2))
        return nn.Sequential(*layers)

    def upblock(self, in_c, out_c):
        return nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, 4, 2, 1),
            nn.BatchNorm2d(out_c),
            nn.ReLU()
        )

    def forward(self, x):
        skips = []
        for layer in self.encoder:
            x = layer(x); skips.append(x)
        skips = skips[:-1][::-1]
        for i, layer in enumerate(self.decoder[:-2]):
            x = layer(x)
            if i < len(skips):
                x = torch.cat([x, skips[i]], 1)
        return self.decoder[-2](x)

# ========= 5) CGAN復元＋分類 → CSV（画像と同じフォルダ） =========
def restore_and_classify():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    gen = UNetGenerator().to(device)
    gen.load_state_dict(torch.load(weight_path, map_location=device))
    gen.eval()

    clf, preprocess, categories = load_classifier(device)
    results = []

    print(f"\n=== CGAN推論＋分類開始 ({dataset}/{attack_type}) ===\n")

    with torch.no_grad():
        for fname in tqdm(sorted(os.listdir(tensor_dir))):
            if not fname.endswith(".pt"):
                continue
            t = torch.load(tensor_dir / fname, map_location=device)
            if t.ndim == 3:
                t = t.unsqueeze(0)

            # [0,1] なら [-1,1] に正規化してGeneratorへ
            t_in = (t - 0.5) / 0.5 if (t.min() >= 0 and t.max() <= 1) else t
            restored = gen(t_in.to(device))
            restored_01 = (restored * 0.5 + 0.5).clamp(0, 1)

            # 分類（必要ならレガシーのResize/Crop含む transforms()）
            x = restored_01.squeeze(0).cpu()
            x = preprocess(x).unsqueeze(0).to(device)
            preds = F.softmax(clf(x), dim=1)
            conf, idx = preds.max(1)
            label = categories[int(idx)]
            confidence = float(conf.item())

            # 画像保存
            base = Path(fname).stem.replace("_wavelet", "")
            out_png = cgan_out_dir / f"{base}_{attack_type}_restored.png"
            save_image(restored_01.cpu(), out_png)

            # 結果記録
            results.append({
                "ImageId": base,
                "CGAN_Label": label,
                "CGAN_Confidence": round(confidence, 6),
            })

    # ✅ CSVは画像と同じフォルダに出力
    pd.DataFrame(results).to_csv(cgan_csv, index=False, encoding="utf-8-sig")
    print(f"✅ CGAN分類結果CSV: {cgan_csv}")
    print(f"✅ 復元画像: {cgan_out_dir}")

# ========= 6) 4段階CSV統合＋成功率 + 失敗画像保存＋失敗一覧CSV（すべて画像と同じフォルダ） =========
def merge_and_evaluate():
    print(f"\n=== 4段階CSV統合＆成功率 ({dataset}/{attack_type}) ===\n")
    df_b  = pd.read_csv(before_csv)
    df_a  = pd.read_csv(after_csv)
    df_w  = pd.read_csv(wavelet_csv)
    df_g  = pd.read_csv(cgan_csv)

    df_b["ImageId"] = df_b["image"].apply(lambda x: Path(str(x)).stem)
    df_a["ImageId"] = df_a["image"].apply(lambda x: Path(str(x)).stem)
    df_g["ImageId"] = df_g["ImageId"].astype(str).apply(
        lambda s: Path(s).stem.replace(f"_{attack_type}_restored","").replace("_restored","").replace("_wavelet","")
    )

    df_b = df_b.rename(columns={"label_name":"label_name_before","confidence":"confidence_before"})
    df_a = df_a.rename(columns={"Advs_label_name":"label_name_after","Advs_confidence":"confidence_after"})

    df = (df_b.merge(df_a, on="ImageId", how="inner")
              .merge(df_w, on="ImageId", how="left")
              .merge(df_g, on="ImageId", how="left"))

    df["AttackSuccess"]            = df["label_name_before"] != df["label_name_after"]
    df["RecoverySuccess_Wavelet"]  = df["label_name_before"] == df.get("Wavelet_Label")
    df["RecoverySuccess_CGAN"]     = df["label_name_before"] == df.get("CGAN_Label")

    # ✅ 統合結果CSVも画像と同じフォルダへ
    df.to_csv(merged_csv, index=False, encoding="utf-8-sig")

    total = len(df)
    atk   = int(df["AttackSuccess"].sum())
    w_ok  = int(df["RecoverySuccess_Wavelet"].sum())
    g_ok  = int(df["RecoverySuccess_CGAN"].sum())
    pct   = lambda n: (n/total*100.0) if total else 0.0

    print("=== 統計結果 ===")
    print(f"総数: {total}")
    print(f"攻撃成功数:        {atk}/{total} ({pct(atk):.2f}%)")
    print(f"Wavelet矯正成功数: {w_ok}/{total} ({pct(w_ok):.2f}%)")
    print(f"CGAN矯正成功数:   {g_ok}/{total} ({pct(g_ok):.2f}%)")
    print(f"\n✅ 出力: {merged_csv}")

    # ---- Wavelet矯正失敗画像の保存 ----
    wavelet_failed = df[df["RecoverySuccess_Wavelet"] == False]
    if len(wavelet_failed) > 0:
        for _, row in wavelet_failed.iterrows():
            img_id = str(row["ImageId"])
            src = wavelet_png_dir / f"{img_id}_wavelet.png"
            if src.exists():
                dst = wavelet_failed_dir / src.name
                copy2(src, dst)
        print(f"❌ Wavelet矯正失敗画像を保存: {wavelet_failed_dir}")
    else:
        print("✅ Wavelet矯正は全件成功")

    # ---- CGAN矯正失敗画像の保存 ----
    cgan_failed = df[df["RecoverySuccess_CGAN"] == False]
    if len(cgan_failed) > 0:
        for _, row in cgan_failed.iterrows():
            img_id = str(row["ImageId"])
            cgan_img_path = cgan_out_dir / f"{img_id}_{attack_type}_restored.png"
            if cgan_img_path.exists():
                dst = cgan_failed_dir / cgan_img_path.name
                copy2(cgan_img_path, dst)
        print(f"❌ CGAN矯正失敗画像を保存: {cgan_failed_dir}")
    else:
        print("✅ CGAN矯正は全件成功")

    # ✅ 失敗一覧CSVは画像と同じフォルダ（cgan_out_dir）へ
    out_cols = [
        "ImageId",
        "label_name_before",
        "label_name_after",
        "Wavelet_Label",
        "CGAN_Label",
        "confidence_before",
        "confidence_after",
        "CGAN_Confidence"
    ]
    failed_concat = pd.concat(
        [
            wavelet_failed.assign(FailStage="Wavelet"),
            cgan_failed.assign(FailStage="CGAN")
        ],
        axis=0, ignore_index=True
    )
    if len(failed_concat) > 0:
        failed_concat[out_cols + ["FailStage"]].to_csv(failed_list_csv, index=False, encoding="utf-8-sig")
        print(f"📝 失敗一覧CSV: {failed_list_csv}")
    else:
        print("📝 失敗一覧CSV: 失敗なしのため出力スキップ")

# ========= 7) 実行 =========
if __name__ == "__main__":
    restore_and_classify()
    merge_and_evaluate()



=== CGAN推論＋分類開始 (data3/CW) ===



100%|██████████| 1000/1000 [00:45<00:00, 22.10it/s]

✅ CGAN分類結果CSV: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore_FGSM_train\data3_CW\pix2pix_class_results_data3_CW.csv
✅ 復元画像: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore_FGSM_train\data3_CW

=== 4段階CSV統合＆成功率 (data3/CW) ===

=== 統計結果 ===
総数: 1000
攻撃成功数:        1000/1000 (100.00%)
Wavelet矯正成功数: 917/1000 (91.70%)
CGAN矯正成功数:   942/1000 (94.20%)

✅ 出力: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore_FGSM_train\data3_CW\compare_data3_CW_Wavelet_CGAN.csv
❌ Wavelet矯正失敗画像を保存: C:\Users\sit\wavelet_CGAN\testdata3\wavelet_restore_failed\data3_CW
❌ CGAN矯正失敗画像を保存: C:\Users\sit\wavelet_CGAN\testdata3\cgan_restore_failed\data3_CW
📝 失敗一覧CSV: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore_FGSM_train\data3_CW\failed_lists_data3_CW.csv


In [10]:
import pandas as pd
from torchvision import models

# === ImageNetクラス名リストをロード ===
weights = models.MobileNet_V2_Weights.IMAGENET1K_V1
imagenet_labels = weights.meta["categories"]

# === CSV読み込み ===
csv_path = r"C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore_FGSM_train\data2_CW\compare_data2_CW_Wavelet_CGAN.csv"
df = pd.read_csv(csv_path)

# === class_XXX → ラベル名に変換 ===
def convert_class_to_label(x):
    if isinstance(x, str) and x.startswith("class_"):
        try:
            idx = int(x.replace("class_", ""))
            return imagenet_labels[idx]
        except:
            return x
    return x

df["label_name_after"] = df["label_name_after"].apply(convert_class_to_label)

# Wavelet / CGAN 列も変換が必要な場合は追加
for col in ["Wavelet_Label", "CGAN_Label"]:
    if col in df.columns:
        df[col] = df[col].apply(convert_class_to_label)

# === 保存（上書き or 別名保存） ===
df.to_csv(csv_path.replace(".csv", "_fixed.csv"), index=False, encoding="utf-8-sig")
print("✅ クラスID → ラベル名変換完了。保存しました。")


✅ クラスID → ラベル名変換完了。保存しました。


In [12]:
import pandas as pd
from torchvision import models

# === ImageNetクラス名リストをロード ===
weights = models.MobileNet_V2_Weights.IMAGENET1K_V1
imagenet_labels = weights.meta["categories"]

# === CSV読み込み ===
csv_path = r"C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore_FGSM_train\data2_CW\compare_data2_CW_Wavelet_CGAN.csv"
df = pd.read_csv(csv_path)

# === class_XXX → ImageNetラベル名に変換 ===
def convert_class_to_label(x):
    if pd.isna(x):
        return x
    if isinstance(x, (int, float)):  # すでに数字（クラスID）ならそのまま対応
        idx = int(x)
        if 0 <= idx < len(imagenet_labels):
            return imagenet_labels[idx]
        return x
    if isinstance(x, str):
        s = x.strip()
        if s.startswith("class_"):
            try:
                idx = int(s.replace("class_", "").strip())
                return imagenet_labels[idx]
            except:
                return s
        elif s.isdigit():
            try:
                idx = int(s)
                return imagenet_labels[idx]
            except:
                return s
        return s
    return x

# === 対象列を一括変換 ===
target_cols = [
    "Original_label_name",
    "Advs_label_name",
    "Wavelet_Label",
    "CGAN_Label"
]

for col in target_cols:
    if col in df.columns:
        df[col] = df[col].apply(convert_class_to_label)

# === 保存（別名保存して上書き防止） ===
out_path = csv_path.replace(".csv", "_label_fixed.csv")
df.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"✅ クラスID → ImageNetラベル名に変換完了\n保存先: {out_path}")


✅ クラスID → ImageNetラベル名に変換完了
保存先: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore_FGSM_train\data2_CW\compare_data2_CW_Wavelet_CGAN_label_fixed.csv


In [7]:
# ==========================================================
# Wavelet＋CGAN 復元Tensorのみ出力（最小・固定フォルダ版）
# 出力形式: sample0001.pt, sample0002.pt, ...
# 出力先: pix2pix_restore_train_v2/<dataset>_<attack_type>/tensor_out/
# 既存ファイルがある場合はスキップ（上書き防止）
# ==========================================================
import os
import sys
import torch
from torch import nn
from tqdm import tqdm
from pathlib import Path

# ========= 1) 設定 =========
dataset     = "data3"        # "data1" / "data2" / "data3"
attack_type = "FGSM"           # "FGSM" / "CW"
BASE        = Path(r"C:\Users\sit\wavelet_CGAN\testdata3")

tensor_dir  = BASE / f"{dataset}_{attack_type}" / "wavelet_eval" / "wavelet_tensor"
weight_path = BASE / "pix2pix_train_v2" / "weights" / "gen_weights_epoch300.pth"

# 出力フォルダ（固定）
tensor_out_dir = BASE / "pix2pix_restore_train_v2" / f"{dataset}_{attack_type}" / "tensor_out"
tensor_out_dir.mkdir(parents=True, exist_ok=True)

# ========= 2) チェック =========
def _must_exist(p: Path, msg: str):
    if not p.exists():
        print(f"[ERROR] {msg}: {p}")
        sys.exit(1)

_must_exist(tensor_dir, "wavelet_tensor フォルダが存在しません")
_must_exist(weight_path, "Generator 重みが見つかりません")

# ========= 3) UNet Generator定義 =========
class UNetGenerator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=64):
        super().__init__()
        self.encoder = nn.Sequential(
            self.block(in_channels, features, normalize=False),
            self.block(features, features * 2),
            self.block(features * 2, features * 4),
            self.block(features * 4, features * 8),
            self.block(features * 8, features * 8),
        )
        self.decoder = nn.Sequential(
            self.upblock(features * 8, features * 8),
            self.upblock(features * 8 * 2, features * 4),
            self.upblock(features * 4 * 2, features * 2),
            self.upblock(features * 2 * 2, features),
            nn.ConvTranspose2d(features * 2, out_channels, 4, 2, 1),
            nn.Tanh(),
        )

    def block(self, in_c, out_c, normalize=True):
        layers = [nn.Conv2d(in_c, out_c, 4, 2, 1)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.LeakyReLU(0.2))
        return nn.Sequential(*layers)

    def upblock(self, in_c, out_c):
        return nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, 4, 2, 1),
            nn.BatchNorm2d(out_c),
            nn.ReLU()
        )

    def forward(self, x):
        skips = []
        for layer in self.encoder:
            x = layer(x)
            skips.append(x)
        skips = skips[:-1][::-1]
        for i, layer in enumerate(self.decoder[:-2]):
            x = layer(x)
            if i < len(skips):
                x = torch.cat([x, skips[i]], 1)
        return self.decoder[-2](x)

# ========= 4) 復元Tensor生成 =========
def restore_tensor_only():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    gen = UNetGenerator().to(device)
    gen.load_state_dict(torch.load(weight_path, map_location=device))
    gen.eval()

    print(f"\n=== Wavelet＋CGAN Tensor生成開始 ({dataset}/{attack_type}) ===\n")
    print(f"出力フォルダ: {tensor_out_dir}\n")

    count = 0
    skipped = 0

    with torch.no_grad():
        for fname in tqdm(sorted(os.listdir(tensor_dir))):
            if not fname.endswith(".pt"):
                continue

            out_path = tensor_out_dir / Path(fname).name
            if out_path.exists():
                skipped += 1
                continue  # 既に存在する場合はスキップ

            # Wavelet tensor 読み込み
            t = torch.load(tensor_dir / fname, map_location=device)
            if t.ndim == 3:
                t = t.unsqueeze(0)
            t_in = (t - 0.5) / 0.5 if (t.min() >= 0 and t.max() <= 1) else t

            # Generatorで復元
            restored = gen(t_in.to(device))
            restored_01 = (restored * 0.5 + 0.5).clamp(0, 1)

            # sample0001.pt の形式で保存
            torch.save(restored_01.cpu(), out_path)
            count += 1

    print(f"\n✅ 新規生成: {count} 件")
    print(f"⏩ スキップ: {skipped} 件（既存ファイル）")
    print(f"📁 出力先: {tensor_out_dir}")

# ========= 5) 実行 =========
if __name__ == "__main__":
    restore_tensor_only()



=== Wavelet＋CGAN Tensor生成開始 (data3/FGSM) ===

出力フォルダ: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore_train_v2\data3_FGSM\tensor_out



100%|██████████| 1000/1000 [00:06<00:00, 143.33it/s]


✅ 新規生成: 1000 件
⏩ スキップ: 0 件（既存ファイル）
📁 出力先: C:\Users\sit\wavelet_CGAN\testdata3\pix2pix_restore_train_v2\data3_FGSM\tensor_out
